<a href="https://colab.research.google.com/github/leekh8/Colaboratory/blob/main/ex09_%EA%B0%9C%2C%EA%B3%A0%EC%96%91%EC%9D%B4_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Colab Notebooks/Colaboratory/AISchool/'

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Colaboratory/AISchool


# 목표
- 개, 고양이 데이터 이미지를 배열로 변환 (이미지 데이터 전처리)
  - 이미지 자체를 저장소에 업로드하는 것 보다 배열로 변환했을 때 용량 소모 적음
- MLP 이미지 데이터 이진 분류
- CNN 이미지 데이터 이진 분류
- 성능 향상을 위한 노력
  - 이미지 증식
  - 전이학습

In [ ]:
# 압축 풀기
from zipfile import ZipFile

# 압축 파일 경로 설정
zip_file = './Data/cats_and_dogs_filtered.zip'

# 압축 파일 해제
with ZipFile(zip_file, 'r') as f:
  f.extractall(path = './Data/')

In [ ]:
# 환경 세팅
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 이미지 다루는 라이브러리
from PIL import Image

In [ ]:
# 1. 각각의 이미지들에 접근 (경로들) - 경로 정리
# train - cats, dogs / test - cats, dogs 경로 변수 저장
train_cats_dir = './Data/cats_and_dogs_filtered/train/cats'
train_dogs_dir = './Data/cats_and_dogs_filtered/train/dogs'

test_cats_dir = './Data/cats_and_dogs_filtered/test/cats'
test_dogs_dir = './Data/cats_and_dogs_filtered/test/dogs'

# os.listdir() - 해당 경로에 있는 파일명 리스트로 출력
train_cats_fn = os.listdir(train_cats_dir)
train_dogs_fn = os.listdir(train_dogs_dir)

test_cats_fn = os.listdir(test_cats_dir)
test_dogs_fn = os.listdir(test_dogs_dir)

train_cats_fn

['cat.0.jpg',
 'cat.1.jpg',
 'cat.10.jpg',
 'cat.100.jpg',
 'cat.101.jpg',
 'cat.102.jpg',
 'cat.103.jpg',
 'cat.104.jpg',
 'cat.105.jpg',
 'cat.106.jpg',
 'cat.107.jpg',
 'cat.108.jpg',
 'cat.109.jpg',
 'cat.11.jpg',
 'cat.110.jpg',
 'cat.111.jpg',
 'cat.112.jpg',
 'cat.113.jpg',
 'cat.114.jpg',
 'cat.115.jpg',
 'cat.116.jpg',
 'cat.117.jpg',
 'cat.118.jpg',
 'cat.119.jpg',
 'cat.12.jpg',
 'cat.120.jpg',
 'cat.121.jpg',
 'cat.122.jpg',
 'cat.123.jpg',
 'cat.124.jpg',
 'cat.125.jpg',
 'cat.126.jpg',
 'cat.127.jpg',
 'cat.128.jpg',
 'cat.129.jpg',
 'cat.13.jpg',
 'cat.130.jpg',
 'cat.131.jpg',
 'cat.132.jpg',
 'cat.133.jpg',
 'cat.134.jpg',
 'cat.135.jpg',
 'cat.136.jpg',
 'cat.137.jpg',
 'cat.138.jpg',
 'cat.139.jpg',
 'cat.14.jpg',
 'cat.140.jpg',
 'cat.141.jpg',
 'cat.142.jpg',
 'cat.143.jpg',
 'cat.144.jpg',
 'cat.145.jpg',
 'cat.146.jpg',
 'cat.147.jpg',
 'cat.148.jpg',
 'cat.149.jpg',
 'cat.15.jpg',
 'cat.150.jpg',
 'cat.151.jpg',
 'cat.152.jpg',
 'cat.153.jpg',
 'cat.154.jpg',
 '

In [ ]:
# 이미지 1장에 접근할 때, 폴더 경로 + 이미지 파일명
tmp_path = os.path.join(train_cats_dir, train_cats_fn[0])
tmp_path

'./Data/cats_and_dogs_filtered/train/cats/cat.0.jpg'

In [ ]:
# 이미지 불러오기 -> 크기 조정 -> 배열로 변환
img = Image.open(tmp_path)
img.resize((224, 224))

np.array(img) # img: image -> np.array로 배열로 바꿈

array([[[203, 164,  87],
        [203, 164,  87],
        [204, 165,  88],
        ...,
        [240, 201, 122],
        [239, 200, 121],
        [238, 199, 120]],

       [[203, 164,  87],
        [203, 164,  87],
        [204, 165,  88],
        ...,
        [241, 202, 123],
        [240, 201, 122],
        [238, 199, 120]],

       [[203, 164,  87],
        [203, 164,  87],
        [204, 165,  88],
        ...,
        [241, 202, 123],
        [240, 201, 122],
        [239, 200, 121]],

       ...,

       [[153, 122,  55],
        [153, 122,  55],
        [153, 122,  55],
        ...,
        [  2,   2,   0],
        [  2,   2,   0],
        [  2,   2,   0]],

       [[152, 121,  54],
        [152, 121,  54],
        [152, 121,  54],
        ...,
        [  2,   2,   0],
        [  2,   2,   0],
        [  2,   2,   0]],

       [[151, 120,  53],
        [151, 120,  53],
        [151, 120,  53],
        ...,
        [  1,   1,   0],
        [  1,   1,   0],
        [  1,   1,   0]]

In [ ]:
len(train_cats_fn)

1000

In [ ]:
# 사진을 불러와 배열로 바꾸는 함수 생성
# 모든 이미지 사이즈는 224, 224 -> array
def load_imgs(folder_path, file_name, img_size = (224, 224)):

  # 이미지들을 묶어줄 빈 리스트 생성
  imgs = [] # append를 통해 변환된 이미지 누적 저장

  for i in file_name:

    # 폴더 경로 + 파일명 합치기
    path = os.path.join(folder_path, i)

    # 파일 오픈 + 리사이즈 (크기 조정)
    img = Image.open(path).resize(img_size)

    # img -> np.array -> imgs 리스트에 뒤로 저장
    imgs.append(np.array(img))

  return np.array(imgs)

In [ ]:
# 각 폴더 연결해서 배열로 변환
X_train_cats = load_imgs(train_cats_dir, train_cats_fn)
X_train_dogs = load_imgs(train_dogs_dir, train_dogs_fn)
X_test_cats = load_imgs(test_cats_dir, test_cats_fn)
X_test_dogs = load_imgs(test_dogs_dir, test_dogs_fn)

# 크기 확인
print(X_train_cats.shape)
print(X_train_dogs.shape)
print(X_test_cats.shape)
print(X_test_dogs.shape)

# 1000장, 224행, 224열, 3채널 (컬러 이미지)

(1000, 224, 224, 3)
(1000, 224, 224, 3)
(500, 224, 224, 3)
(500, 224, 224, 3)


In [ ]:
# train, test 각각 병합
X_train = np.concatenate((X_train_cats, X_train_dogs))
X_test = np.concatenate((X_test_cats, X_test_dogs))

X_train.shape, X_test.shape

((2000, 224, 224, 3), (1000, 224, 224, 3))

In [ ]:
# 답 데이터 생성
# train
# cat: 0(1000), dog: 1(1000)
# test
# cat: 0(500), dog: 1(500)
y_train = np.array([0] * 1000 + [1] * 1000)
y_test = np.array([0] * 500 + [1] * 500)

y_train.shape, y_test.shape

((2000,), (1000,))

In [ ]:
# npz로 저장
np.savez_compressed('./Data/np_cat_vs_dog.npz', X_train = X_train, X_test = X_test, y_train = y_train, y_test = y_test)